In [62]:
import numpy as np
import baseRNN
from datasets import load_dataset
import re
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Dataset

In [174]:
# constants
START_TOKEN = '<START>'
END_TOKEN = '<END>'
NUM_SAMPLES = 1000
imdbDataset = load_dataset("stanfordnlp/imdb")

embeddingsFilepath = '/Users/josep/Desktop/Self/Learning/NLP/RNN/data/glove.6B.300d.txt'

# helper functions
def read_corpus(dataset):
    files = dataset["train"]["text"][:NUM_SAMPLES]
    return [[START_TOKEN] + [re.sub(r'[^\w]', '', w.lower()) for w in f.split(" ")] + [END_TOKEN] for f in files]


def embedding_for_vocab(filepath, words, dimensions):
    vocab_size = len(words)
    embeddings = np.zeros((vocab_size, dimensions))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in words.keys():
                index = words[word]
                embeddings[index] = np.array(vector)[:dimensions]
    return embeddings

imdbCorpus = read_corpus(imdbDataset)

corpusWords = [y for x in imdbCorpus for y in x]
corpusWords = list(set(corpusWords))
word2ind={}
for i in range(len(corpusWords)):
    word2ind[corpusWords[i]] = i

embeddings = embedding_for_vocab(embeddingsFilepath, word2ind, 300)

In [175]:
print(len(corpusWords))

19041


### Training

In [226]:
testRNN = baseRNN.neuralNet(embeddings=embeddings, word2ind=word2ind, outputActivation='softmax',
                            hiddenLayerShapes=[100,100,100], hiddenLayerActivations=['relu', 'relu', 'relu'],
                            lossFunction='crossEntropyLoss', learningRate=.05, epochs=1,
                            adam=False, clipVal=1, debug=False)

In [227]:
overfitCorpus = [imdbCorpus[0][0:20]] * 1000

In [ ]:
testRNN.trainModel(imdbCorpus)

Text #1 - 290 words
Loss: 9.852722842296949
********************************************

Text #2 - 216 words
Loss: 9.851662417563107
********************************************

Text #3 - 95 words
Loss: 9.852690495748211
********************************************

Text #4 - 120 words
Loss: 9.848193979857621
********************************************

Text #5 - 313 words
Loss: 9.851798766764977
********************************************

Text #6 - 125 words
Loss: 9.84223705945431
********************************************

Text #7 - 116 words
Loss: 9.849856651011356
********************************************

Text #8 - 303 words
Loss: 9.846733440440541
********************************************

Text #9 - 482 words
Loss: 9.844346027906175
********************************************

Text #10 - 226 words
Loss: 9.846699317324003
********************************************

Text #11 - 264 words
Loss: 9.846097813940903
********************************************

Text #12 -